In [1]:
import pandas
import databento

In [45]:
pandas.set_option('display.max_rows', 50)
pandas.set_option('display.min_rows', 20)

In [3]:
def get_path(ticker, filename=None):
    ticker = 'nvda'
    #filename = 'xnas-itch-20240105.mbo.dbn.zst'
    return f'/databento/data/mbo/{ticker}/XNAS-20240622-WPPRESG4BH/{filename}'

In [4]:
full_path = get_path('nvda', filename='xnas-itch-20240105.mbo.dbn.zst')
full_path

'/databento/data/mbo/nvda/XNAS-20240622-WPPRESG4BH/xnas-itch-20240105.mbo.dbn.zst'

In [5]:
stored_data = databento.DBNStore.from_file(full_path)
df: pandas.DataFrame = stored_data.to_df().reset_index()

In [6]:
df

,ts_recv,ts_event,rtype,publisher_id,instrument_id,action,side,price,size,channel_id,order_id,flags,ts_in_delta,sequence,symbol
0,2024-01-05 09:00:00.034832298+00:00,2024-01-05 09:00:00.034665350+00:00,160,2,11667,A,B,225.11,41,0,2803,130,166948,274050,NVDA
1,2024-01-05 09:00:00.207384375+00:00,2024-01-05 09:00:00.207209487+00:00,160,2,11667,A,B,478.98,100,0,8955,130,174888,275072,NVDA
2,2024-01-05 09:00:00.214426993+00:00,2024-01-05 09:00:00.214260186+00:00,160,2,11667,A,A,480.78,100,0,12639,130,166807,275517,NVDA
3,2024-01-05 09:00:00.217115284+00:00,2024-01-05 09:00:00.216948664+00:00,160,2,11667,C,A,480.78,100,0,12639,130,166620,275795,NVDA
4,2024-01-05 09:00:00.220495504+00:00,2024-01-05 09:00:00.220328153+00:00,160,2,11667,A,A,483.30,100,0,15775,130,167351,276222,NVDA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4076172,2024-01-05 23:59:49.571847285+00:00,2024-01-05 23:59:49.571680943+00:00,160,2,11667,C,A,491.76,10,0,844161431,130,166342,692007204,NVDA
4076173,2024-01-05 23:59:49.577086028+00:00,2024-01-05 23:59:49.576919786+00:00,160,2,11667,A,A,491.75,10,0,844164463,130,166242,692007207,NVDA
4076174,2024-01-05 23:59:55.956086421+00:00,2024-01-05 23:59:55.955917975+00:00,160,2,11667,T,B,491.73,10,0,0,130,168446,692007319,NVDA
4076175,2024-01-05 23:59:55.956086421+00:00,2024-01-05 23:59:55.955917975+00:00,160,2,11667,F,A,491.73,10,0,844164127,130,168446,692007319,NVDA


In [33]:
df_aux = df.copy()
df_aux['trade_action_fill'] = df_aux['action'].shift(-1)
df_aux['trade_action_cancel'] = df_aux['action'].shift(-2)
df_aux['trade_order_id_fill'] = df_aux['order_id'].shift(-1)
df_aux['trade_order_id_cancel'] = df_aux['order_id'].shift(-2)
df_aux[['action', 'trade_action_fill', 'trade_action_cancel', 'side', 'price', 'size', 'order_id', 'trade_order_id_fill', 'trade_order_id_cancel']]

,action,trade_action_fill,trade_action_cancel,side,price,size,order_id,trade_order_id_fill,trade_order_id_cancel
0,A,A,A,B,225.11,41,2803,8955.0,12639.0
1,A,A,C,B,478.98,100,8955,12639.0,12639.0
2,A,C,A,A,480.78,100,12639,12639.0,15775.0
3,C,A,C,A,480.78,100,12639,15775.0,15775.0
4,A,C,A,A,483.30,100,15775,15775.0,17947.0
...,...,...,...,...,...,...,...,...,...
4076172,C,A,T,A,491.76,10,844161431,844164463.0,0.0
4076173,A,T,F,A,491.75,10,844164463,0.0,844164127.0
4076174,T,F,C,B,491.73,10,0,844164127.0,844164127.0
4076175,F,C,None,A,491.73,10,844164127,844164127.0,NaN


In [41]:
df_aux_trade_action_match_fill = df_aux['trade_action_fill'] == 'F'
df_aux['trade_action_match_fill'] = df_aux_trade_action_match_fill
#df_aux_trade_action_match_fill

In [36]:
df_aux_trade_action_match_cancel = df_aux['trade_action_cancel'] == 'C'
df_aux['trade_action_match_cancel'] = df_aux_trade_action_match_cancel
#df_aux_trade_action_match_cancel

In [37]:
df_aux_trade_action_match_both = df_aux_trade_action_match_fill & df_aux_trade_action_match_cancel
df_aux['trade_action_match_both'] = df_aux_trade_action_match_both
#df_aux_trade_action_match_both

In [38]:
df_aux_trade_action_match_both.value_counts()

False    4007949
True       68228
Name: count, dtype: int64

In [50]:
columns_to_select = ['action', 'trade_action_fill', 'trade_action_match_fill', 'trade_action_cancel',
                     'trade_action_match_cancel', 'trade_action_match_both', 'side', 'price', 'size',
                     'order_id', 'trade_order_id_fill', 'trade_order_id_cancel']

In [51]:
df_aux[columns_to_select]

,action,trade_action_fill,trade_action_match_fill,trade_action_cancel,trade_action_match_cancel,trade_action_match_both,side,price,size,order_id,trade_order_id_fill,trade_order_id_cancel
0,A,A,False,A,False,False,B,225.11,41,2803,8955.0,12639.0
1,A,A,False,C,True,False,B,478.98,100,8955,12639.0,12639.0
2,A,C,False,A,False,False,A,480.78,100,12639,12639.0,15775.0
3,C,A,False,C,True,False,A,480.78,100,12639,15775.0,15775.0
4,A,C,False,A,False,False,A,483.30,100,15775,15775.0,17947.0
5,C,A,False,C,True,False,A,483.30,100,15775,17947.0,17947.0
6,A,C,False,A,False,False,A,483.84,100,17947,17947.0,29299.0
7,C,A,False,C,True,False,A,483.84,100,17947,29299.0,29299.0
8,A,C,False,A,False,False,A,483.57,100,29299,29299.0,29595.0
9,C,A,False,C,True,False,A,483.57,100,29299,29595.0,29595.0


In [49]:
df_aux_matched_rows = df_aux[df_aux['trade_action_match_both'] == True][columns_to_select]
df_aux_matched_rows[df_aux_matched_rows['trade_order_id_fill'] != df_aux_matched_rows['trade_order_id_cancel']] # important that this is empty

,action,trade_action_fill,trade_action_match_fill,trade_action_cancel,trade_action_match_cancel,trade_action_match_both,side,price,size,order_id,trade_order_id_fill,trade_order_id_cancel


In [55]:
more_columns_to_select = columns_to_select + ['order_id_replace']
more_columns_to_select

['action',
 'trade_action_fill',
 'trade_action_match_fill',
 'trade_action_cancel',
 'trade_action_match_cancel',
 'trade_action_match_both',
 'side',
 'price',
 'size',
 'order_id',
 'trade_order_id_fill',
 'trade_order_id_cancel',
 'order_id_replace']

In [80]:
df_aux['order_id_replace'] = 0
df_aux.loc[df_aux['trade_action_match_both'] == True, 'order_id_replace'] = df_aux.loc[df_aux['trade_action_match_both'] == True, 'trade_order_id_fill'].astype(int)
#df_aux.dtypes

In [ ]:
df_aux

In [82]:
df_aux_trades = df_aux[df_aux['action'] == 'T']
df_aux_trades[more_columns_to_select].iloc[0:1000].to_csv('df_aux.csv')

In [84]:
df_aux['order_id'] = df_aux['order_id_replace']

In [ ]:
df_aux

In [88]:
original_columns = df.columns
original_columns

Index(['ts_recv', 'ts_event', 'rtype', 'publisher_id', 'instrument_id',
       'action', 'side', 'price', 'size', 'channel_id', 'order_id', 'flags',
       'ts_in_delta', 'sequence', 'symbol'],
      dtype='object')

In [89]:
df_trades = df_aux_trades[original_columns]
df_trades.to_csv('xnas-itch-20240105.mbo.dbn.trades.csv', index=False)

# Older Code (still important - extracts the Orders)

In [ ]:
df_aux[df_aux['trade_action_fill'] != 'F']

In [15]:
df_trades = df_aux[df_aux['action'] == 'T']
df_trades.to_csv('xnas-itch-20240105.mbo.dbn.trades.csv', index=False)

In [ ]:
from mbo_data_package import filter_df_remove_fill_cancel_pairs

df_orders = filter_df_remove_fill_cancel_pairs(df)

In [23]:
df_orders.to_csv('xnas-itch-20240105.mbo.dbn.orders.csv', index=False)

In [25]:
df_orders.iloc[208176-10:208237+10]

,ts_recv,ts_event,rtype,publisher_id,instrument_id,action,side,price,size,channel_id,order_id,flags,ts_in_delta,sequence,symbol
219128,2024-01-05 14:37:33.797028077+00:00,2024-01-05 14:37:33.796861795+00:00,160,2,11667,A,A,486.33,100,0,48824947,130,166282,44454334,NVDA
219129,2024-01-05 14:37:33.843151345+00:00,2024-01-05 14:37:33.842984915+00:00,160,2,11667,C,B,485.48,25,0,48796979,130,166430,44455569,NVDA
219130,2024-01-05 14:37:33.843153772+00:00,2024-01-05 14:37:33.842987637+00:00,160,2,11667,A,B,485.59,25,0,48826127,130,166135,44455570,NVDA
219131,2024-01-05 14:37:33.866281659+00:00,2024-01-05 14:37:33.866115203+00:00,160,2,11667,A,A,485.95,3,0,48826579,130,166456,44455917,NVDA
219132,2024-01-05 14:37:33.900320009+00:00,2024-01-05 14:37:33.900153604+00:00,160,2,11667,A,A,485.96,25,0,48828083,130,166405,44456813,NVDA
219133,2024-01-05 14:37:33.939642307+00:00,2024-01-05 14:37:33.939470463+00:00,160,2,11667,C,B,485.59,25,0,48826127,0,171844,44459106,NVDA
219134,2024-01-05 14:37:33.939642307+00:00,2024-01-05 14:37:33.939471046+00:00,160,2,11667,A,A,485.94,25,0,48831495,128,171261,44459107,NVDA
219135,2024-01-05 14:37:34.084778199+00:00,2024-01-05 14:37:34.084612230+00:00,160,2,11667,A,B,472.00,20,0,48838395,130,165969,44465026,NVDA
219136,2024-01-05 14:37:34.123896147+00:00,2024-01-05 14:37:34.123730513+00:00,160,2,11667,C,B,485.57,25,0,48815935,130,165634,44465550,NVDA
219137,2024-01-05 14:37:34.133602923+00:00,2024-01-05 14:37:34.133436328+00:00,160,2,11667,C,B,450.96,50,0,9121543,130,166595,44465869,NVDA


In [24]:
!ls -alh

total 521M
drwxr-xr-x  2 ecb ecb 4.0K Jun 25 13:49 .
drwxr-xr-x 15 ecb ecb 4.0K Jun 24 17:31 ..
-rw-r--r--  1 ecb ecb 275K Jun 25 10:21 explore.ipynb
-rw-r--r--  1 ecb ecb  17K Jun 25 13:50 extract_order_book_input.ipynb
-rw-r--r--  1 ecb ecb 123K Jun 24 18:09 order_id_838222951.csv
-rwxr-xr-x  1 ecb ecb 1.3K Jun 24 17:28 read_mbo_data.py
-rw-r--r--  1 ecb ecb 508M Jun 25 13:50 xnas-itch-20240105.mbo.dbn.orders.csv
-rw-r--r--  1 ecb ecb  13M Jun 25 13:25 xnas-itch-20240105.mbo.dbn.trades.csv
